In [1]:
import zipfile
import os
import pandas as pd
import numpy as np

#### 解压缩打包文件 ../data/nyiso/all/raw_data 文件夹

In [2]:
def unzip(source_filename, dest_dir):
    with zipfile.ZipFile(source_filename) as zf:
        try:
            zf.extractall(dest_dir)
        except:
            print(source_filename)
            return


In [3]:
zips = []
for file in os.listdir("../data/nyiso/zips"):
    if file.endswith(".zip"):
        zips.append(file)
for z in zips:
    try:
        unzip('../data/nyiso/zips/' + z, '../data/nyiso/all/raw_data')
        print('../data/nyiso/zips/' + z + "extract done!")
    except:
        print('./data/nyiso/zips/' + z)
        continue

../data/nyiso/zips/20120601pal_csv.zipextract done!
../data/nyiso/zips/20031001pal_csv.zipextract done!
../data/nyiso/zips/20020801pal_csv.zipextract done!
../data/nyiso/zips/20121001pal_csv.zipextract done!
../data/nyiso/zips/20080301pal_csv.zipextract done!
../data/nyiso/zips/20030801pal_csv.zipextract done!
../data/nyiso/zips/20150601pal_csv.zipextract done!
../data/nyiso/zips/20070401pal_csv.zipextract done!
../data/nyiso/zips/20101101pal_csv.zipextract done!
../data/nyiso/zips/20130601pal_csv.zipextract done!
../data/nyiso/zips/20140401pal_csv.zipextract done!
../data/nyiso/zips/20101001pal_csv.zipextract done!
../data/nyiso/zips/20050201pal_csv.zipextract done!
../data/nyiso/zips/20090701pal_csv.zipextract done!
../data/nyiso/zips/20111101pal_csv.zipextract done!
../data/nyiso/zips/20100601pal_csv.zipextract done!
../data/nyiso/zips/20100801pal_csv.zipextract done!
../data/nyiso/zips/20050801pal_csv.zipextract done!
../data/nyiso/zips/20141201pal_csv.zipextract done!
../data/nyis

../data/nyiso/zips/20110401pal_csv.zipextract done!
../data/nyiso/zips/20090201pal_csv.zipextract done!
../data/nyiso/zips/20060701pal_csv.zipextract done!
../data/nyiso/zips/20150201pal_csv.zipextract done!
../data/nyiso/zips/20130201pal_csv.zipextract done!
../data/nyiso/zips/20151201pal_csv.zipextract done!
../data/nyiso/zips/20040801pal_csv.zipextract done!
../data/nyiso/zips/20051201pal_csv.zipextract done!
../data/nyiso/zips/20131001pal_csv.zipextract done!
../data/nyiso/zips/20070301pal_csv.zipextract done!
../data/nyiso/zips/20040501pal_csv.zipextract done!
../data/nyiso/zips/20090901pal_csv.zipextract done!
../data/nyiso/zips/20140801pal_csv.zipextract done!
../data/nyiso/zips/20151001pal_csv.zipextract done!
../data/nyiso/zips/20100501pal_csv.zipextract done!
../data/nyiso/zips/20141101pal_csv.zipextract done!
../data/nyiso/zips/20011101pal_csv.zipextract done!
../data/nyiso/zips/20110201pal_csv.zipextract done!


##### 所有文件整合到合并文件里：  combined_nyiso.csv

In [4]:
csvs = []
for file in os.listdir("../data/nyiso/all/raw_data"):
    if file.endswith("pal.csv"):
        csvs.append(file)

In [6]:
fout=open("../data/nyiso/all/combined_nyiso.csv","a")

# 第一个文件咱们保存头部column name信息:
for line in open("../data/nyiso/all/raw_data/"+csvs[0]):
    fout.write(line)
# 后面的部分可以跳过 headers:    
for file in csvs[1:]:
    f = open("../data/nyiso/all/raw_data/"+file)
    f.__next__() # 跳过 header
    for line in f:
         fout.write(line)
    f.close() # 关闭文件
fout.close()

#### 清洗和了解数据

In [7]:
df = pd.read_csv('../data/nyiso/all/combined_nyiso.csv')

In [9]:
cols = df.columns
cols

Index(['Time Stamp', 'Time Zone', 'Name', 'PTID', 'Load'], dtype='object')

In [10]:
df.columns = [col.lower().replace(' ', '') for col in cols]
df = df[['timestamp', 'name', 'ptid', 'load']]

In [11]:
df.to_csv('../data/nyiso/all/combined_iso.csv', index=False)

In [17]:
df.name.unique()

array(['CAPITL', 'CENTRL', 'DUNWOD', 'GENESE', 'HUD VL', 'LONGIL',
       'MHK VL', 'MILLWD', 'N.Y.C.', 'NORTH', 'WEST', 'N.Y.C._LONGIL'], dtype=object)

#### 建立天气站/weather stations的映射关系
我们的数据帧里区域的名称已经有了，我们要建立起它们和对应的城市还有天气站之间的映射关系

In [26]:
regions = list(df.name.unique())[:-1]
regions

['CAPITL',
 'CENTRL',
 'DUNWOD',
 'GENESE',
 'HUD VL',
 'LONGIL',
 'MHK VL',
 'MILLWD',
 'N.Y.C.',
 'NORTH',
 'WEST']

In [27]:
region_names = ['Capital', 'Central', 'Dunwoodie', 'Genese', 'Hudson Valley', 'Long Island', 'Mohawk Valley', 'Millwood', 'NYC', 'North', 'West']
cities = ['Albany', 'Syracuse', 'Yonkers', 'Rochester', 'Poughkeepsie', 'Huntington', 'Utica', 'Yorktown', 'NYC', 'Plattsburgh', 'Buffalo']
weather_stations = ['kalb', 'ksyr', 'kyon', 'kroc', 'kpou', 'khun', 'kuti', 'kyor', 'kjfk', 'kpbg', 'kbuf']

In [28]:
weather_dict = dict(zip(regions, zip(weather_stations, region_names, cities)))
weather_dict

{'CAPITL': ('kalb', 'Capital', 'Albany'),
 'CENTRL': ('ksyr', 'Central', 'Syracuse'),
 'DUNWOD': ('kyon', 'Dunwoodie', 'Yonkers'),
 'GENESE': ('kroc', 'Genese', 'Rochester'),
 'HUD VL': ('kpou', 'Hudson Valley', 'Poughkeepsie'),
 'LONGIL': ('khun', 'Long Island', 'Huntington'),
 'MHK VL': ('kuti', 'Mohawk Valley', 'Utica'),
 'MILLWD': ('kyor', 'Millwood', 'Yorktown'),
 'N.Y.C.': ('kjfk', 'NYC', 'NYC'),
 'NORTH': ('kpbg', 'North', 'Plattsburgh'),
 'WEST': ('kbuf', 'West', 'Buffalo')}

In [31]:
df.tail()

,timestamp,name,ptid,load
17932770,04/24/2012 23:55:00,MHK VL,61756.0,723.1
17932771,04/24/2012 23:55:00,MILLWD,61759.0,244.9
17932772,04/24/2012 23:55:00,N.Y.C.,61761.0,4829.8
17932773,04/24/2012 23:55:00,NORTH,61755.0,710.3
17932774,04/24/2012 23:55:00,WEST,61752.0,1697.9


In [2]:
df = pd.read_csv('../data/nyiso/all/combined_iso.csv')

In [3]:
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%m/%d/%Y %H:%M:%S')

In [6]:
df.head(100)

,timestamp,name,ptid,load
0,2011-09-20 00:00:00,CAPITL,61757.0,1003.8
1,2011-09-20 00:00:00,CENTRL,61754.0,1439.3
2,2011-09-20 00:00:00,DUNWOD,61760.0,513.7
3,2011-09-20 00:00:00,GENESE,61753.0,910.5
4,2011-09-20 00:00:00,HUD VL,61758.0,852.9
5,2011-09-20 00:00:00,LONGIL,61762.0,1942.3
6,2011-09-20 00:00:00,MHK VL,61756.0,620.2
7,2011-09-20 00:00:00,MILLWD,61759.0,271.6
8,2011-09-20 00:00:00,N.Y.C.,61761.0,4909.7
9,2011-09-20 00:00:00,NORTH,61755.0,675.2


In [7]:
df_sorted = df.sort_values(by=['timestamp'])

In [8]:
df_sorted.head(100)

,timestamp,name,ptid,load
17865851,2001-05-26 00:00:00,HUD VL,61758.0,970.0
17865856,2001-05-26 00:00:00,WEST,61752.0,1505.0
17865855,2001-05-26 00:00:00,NORTH,61755.0,659.0
17865847,2001-05-26 00:00:00,CAPITL,61757.0,985.0
17865848,2001-05-26 00:00:00,CENTRL,61754.0,1461.0
17865849,2001-05-26 00:00:00,DUNWOD,61760.0,443.0
17865850,2001-05-26 00:00:00,GENESE,61753.0,830.0
17865854,2001-05-26 00:00:00,N.Y.C._LONGIL,NaN,6217.0
17865852,2001-05-26 00:00:00,MHK VL,61756.0,623.0
17865853,2001-05-26 00:00:00,MILLWD,61759.0,166.0


In [14]:
df_sorted.to_csv('../data/nyiso/all/combined_iso_sorted.csv', index=False)

In [15]:
df_sorted_north.head()

,timestamp,name,ptid,load
17865855,2001-05-26 00:00:00,NORTH,61755.0,659.0
17865865,2001-05-26 00:00:50,NORTH,61755.0,636.0
17865875,2001-05-26 00:05:20,NORTH,61755.0,649.0
17865885,2001-05-26 00:06:50,NORTH,61755.0,646.0
17865895,2001-05-26 00:11:50,NORTH,61755.0,647.0


In [16]:
df_sorted_north2 = df_sorted_north[df_sorted_north.timestamp > pd.to_datetime('2012-10-27')]
df_sorted_north2 = df_sorted_north2[df_sorted_north2.timestamp < pd.to_datetime('2016-01-01')]
df_sorted_north2.head()

,timestamp,name,ptid,load
8470880,2012-10-27 00:05:00,NORTH,61755.0,700.7
8470891,2012-10-27 00:10:00,NORTH,61755.0,694.7
8470902,2012-10-27 00:15:00,NORTH,61755.0,692.8
8470913,2012-10-27 00:20:00,NORTH,61755.0,686.5
8470924,2012-10-27 00:25:00,NORTH,61755.0,685.3


In [17]:
df_sorted_north2.head(300)

,timestamp,name,ptid,load
8470880,2012-10-27 00:05:00,NORTH,61755.0,700.7
8470891,2012-10-27 00:10:00,NORTH,61755.0,694.7
8470902,2012-10-27 00:15:00,NORTH,61755.0,692.8
8470913,2012-10-27 00:20:00,NORTH,61755.0,686.5
8470924,2012-10-27 00:25:00,NORTH,61755.0,685.3
8470935,2012-10-27 00:30:00,NORTH,61755.0,677.6
8470946,2012-10-27 00:35:00,NORTH,61755.0,681.2
8470957,2012-10-27 00:40:00,NORTH,61755.0,676.9
8470968,2012-10-27 00:45:00,NORTH,61755.0,673.4
8470979,2012-10-27 00:50:00,NORTH,61755.0,682.4


In [18]:
df_sorted_north2.tail()

,timestamp,name,ptid,load
7749594,2015-12-31 23:35:00,NORTH,61755.0,495.0
7749605,2015-12-31 23:40:00,NORTH,61755.0,484.9
7749616,2015-12-31 23:45:00,NORTH,61755.0,489.4
7749627,2015-12-31 23:50:00,NORTH,61755.0,486.4
7749638,2015-12-31 23:55:00,NORTH,61755.0,494.9


In [19]:
df_sorted_north2.to_csv('../data/nyiso/all/north.csv')

In [20]:
df_sorted_north2.shape

(338795, 4)